# CA 02: Genetic algorithms
## Pardis Zandkarimi -
Introduction: In this CA we learnt about basics of genetic algorithms for finding the optimal solution in a large range of possibilities. The portfolio is being modeled and we are looking for the best weights to pick from each assets.

Chromosome: Chromosomes definition can vary problem to problem. In this case chromosome is a list of weights that we picked from each asset respectively.
Algorithm: first of all we import the libraries.

In [63]:
import pandas as pd
import copy
import numpy as np
import scipy as sc
from random import randrange

Then we define values that are going to be used.

In [64]:
df = pd.read_csv('data.csv')
num_assets = len(df)
diversity_weight = 0.3
return_weight = 120
risk_weight = 15
max_generation_iteration = 200
crossover_rate = 0.7
population_size = 300
mutation_rate = 0.2

First of all we need to produce our first generation randomly to make them improve in the next iterations. So this function is defined. It produces a matrix of random numbers between zero and one and normalize each row to be sure that sum of each chromosome will be equal to one.

In [65]:
def first_generation_genertor():
  pop = np.random.random(size=(population_size, num_assets))

  # ensure the sum of each individual is 1(normaliza)
  for i in pop:
    i /= np.sum(i)
  return pop


Then we define fitness function. This function is going to tell us how good is a chromosome(based ofn the values that are important for us (low risk, high return and high diversity)

In [66]:
#negative fitness is possible
def fitness(chromosome):
  diversity_value = np.count_nonzero(chromosome)
  return_value = np.sum(np.multiply(df['return'], chromosome))
  risk_value = np.sum(np.multiply(df['risk'], chromosome))
  return ( (return_value )**2 /
          (risk_value ))

This function prints the results of the generation.

In [67]:
def print_generation_info(fitness_list, iteration):
  print(f"max fitness of the generation: {max(fitness_list)}")
  print(f"index of the best chromosome: {np.argmax(fitness_list)}")
  print(f"min fitness of the generation: {min(fitness_list)}")
  print(f"average fitness of the generation: {np.average(fitness_list)}")
  print(f"generation : {iteration}")
  print("----------------------------")
  return np.argmax(fitness_list)

First of all we need to choose two parents from the population to crossover. We want the better individuals to have a higher chance of being selected by roulette wheel. So first of all we give each individual a number (based on their fitness ranking) and then select two of them randomly. I first didnt use ranking and the average fitness of the generations didn't vary much trhough iterations so I used the ranking technique.

In [68]:
def choose_parents(population, pop_fitness):
  selection_probs = []
  rank = sc.stats.rankdata(pop_fitness,  method='min')
  rank = rank /np.sum(rank)

  parent1_index = np.random.choice(len(population), p=rank)
  parent2_index = np.random.choice(len(population), p=rank)
  return population[parent1_index], population[parent2_index]

After we choose the parents we need to produce new generation from them so we crossover. This crossover is single point but the point is being chosen randomly. After we crossover the chromosomes normaliza each chromosome to be sure that sum of assets is going to be equal to one.

In [69]:
def crossover(parent1, parent2, crossover_rate):
  if(np.random.random() > crossover_rate):
    return parent1, parent2

  crossover_point = randrange(0, len(parent1))
  new_chromosome1 = np.empty_like(parent1)
  new_chromosome2 = np.empty_like(parent1)
  
  new_chromosome1[:crossover_point] = parent1[:crossover_point]
  new_chromosome1[crossover_point:]= parent2[crossover_point:]

  new_chromosome2[:crossover_point] = parent2[:crossover_point]
  new_chromosome2[crossover_point:] = parent1[crossover_point:]
   
  new_chromosome1 /= np.sum(new_chromosome1)
  new_chromosome2 /= np.sum(new_chromosome2)
  
  return new_chromosome1, new_chromosome2

After the crossover we mutate the chromosomes by this function:

In [70]:
def mutation(mutation_probability, chromosome):
  for i in range (num_assets):
    n = np.random.random()
    if(n < mutation_probability):
      chromosome[i] *= np.random.randint(5)
  #normalization
  chromosome /= np.sum(chromosome)   
  return chromosome


In each generation we need to check if the goal chromosome has been found or not. This function checks if a chromosome satisfied the conditions of the problem.

In [71]:
def goal_test(chromosome):
  return_condition = np.sum(np.multiply(df['return'], chromosome)) >= 10
  
  diversity_condition = np.count_nonzero(chromosome) >= 30
  risk_condition = np.sum(np.multiply(df['risk'], chromosome)) <= 0.6
  
  if(return_condition and diversity_condition and risk_condition):
    print("goal state!")
    return True

After that all is left is that to call the functions. We produce generations and for each generation produce new chromosomes to form a new generation.

In [72]:
genaration_iteration = 1
first_population = first_generation_genertor()
first_generations_fitness = list(map(fitness, first_population))
best_chromosome_index = print_generation_info(first_generations_fitness, 0)
goal = False

num_assets = len(df)
pop = np.random.random(size=(2, num_assets))
for i in pop:
   i /= np.sum(i)

while((genaration_iteration <= max_generation_iteration) & (goal == False)):
  new_generation = []
  for i in range (int(population_size/2)):
    parent1, parent2 = choose_parents(first_population, first_generations_fitness)
    child1 = []
    child2 = []
    child1, child2 = crossover(parent1, parent2, crossover_rate)
    child1 = mutation(mutation_rate, child1)
    child2 = mutation(mutation_rate, child2)
    
    new_generation.append(child1)
    new_generation.append(child2)
    
    
  
  new_generation_fitness = list(map(fitness, new_generation))
  best_chromosome_index = print_generation_info(new_generation_fitness, genaration_iteration)
  for i in first_population:
    if goal_test(i):
      print(f"It took {genaration_iteration} generation iterations to reach the goal state")
      goal = True
      break
  
  #chenge the generation(update first geeration)
  first_population = copy.deepcopy(new_generation)
  first_generations_fitness = new_generation_fitness
  genaration_iteration += 1


max fitness of the generation: 1.385493498889347
index of the best chromosome: 134
min fitness of the generation: 1.0555694460170226
average fitness of the generation: 1.2041623144586129
generation : 0
----------------------------
max fitness of the generation: 1.6640976077054097
index of the best chromosome: 201
min fitness of the generation: 0.9409118372547182
average fitness of the generation: 1.2493017182115371
generation : 1
----------------------------
max fitness of the generation: 2.1715355068116518
index of the best chromosome: 117
min fitness of the generation: 0.9922444453570709
average fitness of the generation: 1.3057217810219404
generation : 2
----------------------------
max fitness of the generation: 4.059270181385992
index of the best chromosome: 276
min fitness of the generation: 0.8381042138514334
average fitness of the generation: 1.4375363559920538
generation : 3
----------------------------
max fitness of the generation: 4.725938377884031
index of the best chromos

# Questions:

## 1. What is the mpact of the very large or very small population size?
If the population is too large it takes long to produce each generation. But if it's too small it takes lots of iterations to find the optimal answer cause variation of chromosome combinations are so low.

## 2. What is the impact of increasing the population size on the accuracy and the time complexity?
It takes longer and longer to produce the generation in each iteration. But It is more accurate cause we can have more fit chidren from the chosen fit parents.

## 3. What is the impact of crossover and mutation? Compare. Is it possible to use only one of them?
Crossover makes new chromosomes from a parent. Its important cause we are more likely to choose parents with the most fitness values so the combination of two fit parents might lead us to the answer. But mutation is just as important as crossover is. We need to generate some of the gens randomly cause in some cases crossover does not lead us to the answer and no one of the parents have the gen that we need to reach the goal state.

## 4. What are other techniques to reach the goal state faster in this case?
Using elit population in the next generation can be helpful. We can also try techniques other than roulette wheel.

## 5. Sometimes chromosomes dont change during a generation iteration. Why this issue happens? How can it be solved?
It happens if the mutation rate value is so low and the diversity of the population is not as much as we are expecting it to be and chromosomes are so alike. In this case we get stuck suboptimal situation and it takes lots of generations to reach the goal state. By increasing mutation rate or using other techniques in crossover (such as uniform crossover) this issue can be solved.

## 6. What is your suggestion about ending the programm when there is no possible way to goal state?
To assign a number to max iteration value we can prevent getting stuck in a never ending loop.